<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/master/04-Deep-Learning/notebooks/P4-MLP_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clasificación y Regresión con redes MLP y CNN</h1>

En esta notebook practicaremos el uso de las redes CNN en varios conjuntos de datos y realizaremos algunas comparaciones con las redes MLP.

Verifiquemos que el entorno de ejecución en Colab sea GPU

In [ ]:
import tensorflow as tf

print('GPU presente en: {}'.format(tf.test.gpu_device_name()))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

# 1. MNIST Fashion

## El conjunto de datos

This is a dataset of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images. This dataset can be used as a drop-in replacement for MNIST. The class labels are:
 <ul>
  <li>0 T-shirt/top</li>
  <li>1 Trouser</li>
  <li>2 Pullover</li>
  <li>3 Dress</li>
  <li>4 Coat</li>
  <li>5 Sandal</li>
  <li>6 Shirt</li>
  <li>7 Sneaker</li>
  <li>8 Bag</li>
  <li>9 Ankle boot</li>
</ul>

Creamos un diccionario con los nombres de las clases

In [ ]:
prendas_list = ['Camiseta','Pantalones','Suéter','Vestido','Abrigo','Sandalia','Camisa','Sneaker','Bolsa','Botín']

prendas = dict(enumerate(prendas_list))
prendas

Cargamos del dataset

In [ ]:
from keras.datasets import fashion_mnist

(X_train, y_train_classes), (X_test, y_test_classes) = fashion_mnist.load_data()

print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train_classes.shape}")
print(f"X test shape: {X_test.shape}")
print(f"y test shape: {y_test_classes.shape}")

Veamos la distribución de clases

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

clases, conteos = np.unique(y_train_classes, return_counts=True)

plt.figure(figsize=(6,4),dpi=120)
plt.bar(clases, conteos)
plt.xticks(clases,list(prendas.values()), rotation=90, ha='center')
plt.title("Distribución de clases")
plt.show()

## 🟢 Preprocesamiento y Separación

Obtenemos las clases como vectores *one-hot*. A partir de aquí, tenemos dos versiones de las etiquetas:

* `y_train_classes`, `y_test_classes`: Vectores de clases, cada etiqueta es un número entero positivo indicando la clase a la que pertenece la instancia.
* `y_train`, `y_test`: Matrices de clases, cada etiqueta es un vector *one-hot* indicando a qué clase pertenece cada instancia.

Parte de la práctica es saber cuándo usar cada versión.

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = 10
y_train = to_categorical(y_train_classes, num_classes)
y_test = to_categorical(y_test_classes, num_classes)

Re-escalamos

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Dividimos en train-validation-split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  stratify=y_train_classes,
                                                  test_size=0.15,
                                                  random_state=42)

print('Train size:', X_train.shape)
print('Validation size:', X_val.shape)
print('Test size:', X_test.shape)

Visualicemos algunas imágenes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# tamaño del conjunto de instancias
m = y_train.shape[0]

# Elegimos algunas instancias al azar para mostrar
random_idxs = np.random.choice(m, 36, replace=False)
images = X_train[random_idxs, :, :].reshape((6,6,28,28))
images_labels = y_train_classes[random_idxs].reshape((6,6))

# visualización de las imágenes
fig, axs = plt.subplots(6,6,figsize=(8,6))
for i in range(6):
    for j in range(6):
        axs[i,j].imshow(images[i,j], cmap='Greys')
        axs[i,j].set_title(prendas[images_labels[i,j]], fontsize=9)
        axs[i,j].axis('off')
fig.show()

## ⭕ Práctica 1

* Entrenar una red MLP para clasificar este conjunto de entrenamiento. Puedes escoger libremente el número de capas, neuronas, funciones de activación, optimizador y épocas. Usa la métrica accuracy. Ten cuidado con el sobre-entrenamiento. **Recuerda que la capa de entrada y la de salida están fijas**. No olvides incluir una capa flatten.
* Grafica las curvas de entrenamiento.
* Reportar el accuracy y la función de pérdida en el conjunto de prueba. Para esto, usa el método `evaluate`.
* Muestra la matriz de confusión.

Matriz de confusión

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input

model_mlp = ...

🟢 Grafica la matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix

y_hat = model_mlp.predict(X_test_flatten)

y_pred = np.argmax(y_hat, axis=1)

conf_matrix = confusion_matrix(y_test_classes, y_pred)

plt.figure(figsize=(6,4),dpi=120)
plt.imshow(conf_matrix)
plt.xticks(list(prendas.keys()),list(prendas.values()), rotation=90, ha='center')
plt.yticks(list(prendas.keys()),list(prendas.values()))
plt.title("Confusion Matrix")
plt.colorbar()
plt.show()

## ⭕ Práctica 2

* Entrenar una red MLP para clasificar este conjunto de entrenamiento. Puedes escoger libremente el número de capas, neuronas, funciones de activación, optimizador y épocas. Usa la métrica accuracy. Ten cuidado con el sobre-entrenamiento. **Recuerda que la capa de entrada y la de salida están fijas**.
* Grafica las curvas de entrenamiento.
* Reportar el accuracy y la función de pérdida en el conjunto de prueba. Para esto, usa el método `predict`.
* Muestra la matriz de confusión.
* Compara el rendimiento de la MLP y la CNN.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Input

model_cnn = ...

In [ ]:
from sklearn.metrics import confusion_matrix

y_hat = model_cnn.predict(X_test)

y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_hat, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,4),dpi=120)
plt.imshow(conf_matrix)
plt.xticks(list(prendas.keys()),list(prendas.values()), rotation=90, ha='center')
plt.yticks(list(prendas.keys()),list(prendas.values()))
plt.title("Confusion Matrix")
plt.colorbar()
plt.show()

# ⭕ Práctica 3: Predicción de edad

En este dataset usarás una red CNN para un problema de regresión con imágenes. El problema consiste en predecir la edad de individuos a partir de una imagen de sus rostros. Para esto, usaremos una versión preprocesada del dataset UTKFace.

[Fuente del dataset](https://www.kaggle.com/datasets/moritzm00/utkface-cropped/data)

🟢 Descargar el dataset

In [ ]:
!gdown 1GWgB_91PuHRjzorZpTUZxRbeIW85TlbZ

🟢 Descomprimir el dataset

In [ ]:
from zipfile import ZipFile

with ZipFile('UTKFace_dataset.zip', 'r') as zipObj:
   zipObj.extractall()
print("Extracción finalizada")

In [ ]:
#@title Clase para un generador con etiquetas continuas

import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class AgeRegressionGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, datagen=None, batch_size=32, target_size=(224, 224), shuffle=True):
        self.directory = directory
        self.datagen = datagen
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle

        # Solo guardar nombres de archivos y edades, NO las imágenes
        self.filenames = []
        self.ages = []

        for filename in os.listdir(directory):
            if filename.endswith('.jpg'):
                age = int(filename.split('_')[0])
                self.filenames.append(filename)
                self.ages.append(age)

        self.indices = np.arange(len(self.filenames))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.filenames) / self.batch_size))

    def __getitem__(self, index):
        # Calcular índices del batch
        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, len(self.filenames))
        batch_indices = self.indices[start:end]

        # Cargar solo las imágenes de este batch
        batch_x = np.zeros((len(batch_indices), *self.target_size, 3), dtype=np.float32)
        batch_y = np.zeros(len(batch_indices), dtype=np.float32)

        for i, idx in enumerate(batch_indices):
            # Cargar imagen solo cuando se necesita
            img_path = os.path.join(self.directory, self.filenames[idx])
            img = Image.open(img_path).convert('RGB')
            img = img.resize(self.target_size)
            img_array = np.array(img, dtype=np.float32)

            # Aplicar transformaciones si existe datagen
            if self.datagen:
                img_array = self.datagen.random_transform(img_array)
                img_array = self.datagen.standardize(img_array)
            else:
                img_array = img_array / 255.0

            batch_x[i] = img_array
            batch_y[i] = self.ages[idx]

        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)




🟢 Definimos los generadores de imágenes

In [ ]:
# Definimos las rutas de los archivos

base_path = "UTKFace"

train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')
val_path = os.path.join(base_path, 'val')

# Crear generadores
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2]
    )

train_generator = AgeRegressionGenerator(
    train_path,
    datagen=train_datagen,
    batch_size=32,
    shuffle=True
    )

val_generator = AgeRegressionGenerator(
    val_path,
    datagen=None,  # Sin aumento para validación
    batch_size=32,
    shuffle=False # Sin reordenar en cada pasada
    )

test_generator = AgeRegressionGenerator(
    test_path,
    datagen=None, # Sin aumento para prueba
    batch_size=32,
    shuffle=False # Sin reordenar en cada pasada
    )

# Longitud de los generadores
print(f"Train batches: {len(train_generator)}")
print(f"Val batches: {len(val_generator)}")
print(f"Test batches: {len(test_generator)}")

🟢 Veamos algunos ejemplos y la forma de cada lote salido del generador

In [ ]:
import matplotlib.pyplot as plt

sample_x, sample_y = train_generator[0]
print(f"Batch shape: {sample_x.shape}, Ages shape: {sample_y.shape}")
print(f"Sample ages: {sample_y[:5]}")

plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(sample_x[i])
    plt.title(f"Age: {int(sample_y[i])}")
    plt.axis('off')
plt.show()

🔴 Define una red convolucional para este problema. Usa la función de perdida `mse` y la métrica `mae`.

La entrada de la red serán tensores $224\times 224\times 3$.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from keras.models import Sequential

model = ...

🔴 Compila el modelo e imprime el `summary()`

In [ ]:
from keras.optimizers import Adam



🔴 Incluye un callback `EarlyStopping`. Elige los hiperparámetros que consideres

In [ ]:
from keras.callbacks import EarlyStopping



🔴 Realiza el entrenamiento durante un número de épocas que consideres apropiado. Usa `validation_steps=val_steps`. Usa un callback EarlyStopping.

El objetivo es que el modelo no haga overfitting.

In [ ]:
val_steps = len(val_generator) // 4

# Entrenar modelo
history = ...

🟢 Muestra las curvas de perdida y métrica

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(11,5))

plt.subplot(1,2,1)
plt.title("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()

plt.subplot(1,2,2)
plt.title("Validation and Training MAE",fontsize=14)
plt.plot(history.history['mae'], label='train')
plt.plot(history.history['val_mae'], label='validation')
plt.legend()

plt.show()

🟢 Salva el modelo

In [ ]:
# Guardar modelo
model.save('nombre_del_modelo.keras')

🔴 Evalua el modelo en el conjunto de prueba con el método `evaluate` del modelo.

🟢 Grafica las predicciones contra las edades reales para visualizar los errores

In [ ]:
import matplotlib.pyplot as plt

y_test = np.array(test_generator.ages)
y_pred = model.predict(test_generator)

plt.figure()
plt.scatter(y_test, y_pred)
plt.xlabel("True Age")
plt.ylabel("Predicted Age")
plt.title("True vs Predicted Ages")
plt.show()

🔴 **OPCIONAL**: Toma alguna fotografía tuya o de quién prefieras y pasala por el modelo para que prediga la edad.

Puedes probar con tu modelo entrenado. Otra opción es probar con un modelo que ya he entrenado, en cuyo caso, descomenta las líneas de la siguiente celda.

In [ ]:
from keras.models import load_model

# !gdown 1mnRGVMqRh0-0u7YRLpEws6iO5RCLgLoZ

# model = load_model('age_regression_model.keras')
# model.summary()

Función para hacer predicciones de edad con el modelo

In [ ]:
def predict_age(model, img_path, img_size=(224, 224)):
    """Predecir edad de una imagen"""
    image = Image.open(img_path).convert('RGB')
    image = image.resize(img_size)
    img_array = np.array(image, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predicted_age = model.predict(img_array)[0][0]
    return predicted_age

Especifica la ruta de la imagen. Puedes subirla a la sección de archivos de colab

In [ ]:
img_path = '...ruta_del_archivo...'

In [ ]:
predicted_age = predict_age(model, img_path)
print(f"Edad predicha: {predicted_age:.1f} años")